# Seek Market Analysis
#### Collaborators:
James Rutherford - s5132147 - james.rutherford@griffithuni.edu.au  
Zakaria Wardere - s5302830 - zakaria.wardere@griffithuni.edu.au

## Part 2

C:\Users\jruth\AppData\Local\Temp\ipykernel_15260\1545033640.py:5: DtypeWarning: Columns (0,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data.csv')
